# LeNet-5

# Tensorflow - tf.contrib.learn

Neste tutorial utilizaremos o framework **tf.contrib.learn**, que permite construir de forma fácil os modelos de Deep learning utilizando o API de alto nível [Estimator](https://www.tensorflow.org/api_guides/python/contrib.learn#estimators). Estimator oferece classes que você pode instanciar para configurar rapidamente tipos de modelos de classificadores.

## Exemplo 1

Neste exemplo vamos criar um rede Deep Learning para reconhecer digitos (Base de dados MNIST) e da um rótulo correto para cada imagem de dígito).

### DATA MNIST

Os dados do MNIST estão no site de [Yann LeCun](http://yann.lecun.com/exdb/mnist/). Os dados estão divididos em 3 partes:

* **Treinamento**: 55,000 dados
* **Teste**: 10,000 dados
* **Validação**: 5,000 dados

Os dados são imagens de 28x28 pixels de 1 canal, o seguinte código em python visualiza um elemento do dataset:

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np
from Lenet5_model import get_data_mnist

train_data , train_labels , eval_data , eval_labels = get_data_mnist()

In [ ]:
img=np.reshape(train_data[15],(28,28))
plt.imshow(img)

## LeNet-5 

A estrutura da rede neural convolutiva (LeNet-5) proposta por [Yann lecun,1998](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf) é apresentada na Figura:

<img src="img/lenet5.png">

O arquivo [Lenet5_model.py](../../edit/Examples/Lenet5_model.py) contém o código principal do modelo, vamos configurar uma versão similar da rede LeNet-5.

Se pede contruir a siguente configuração de rede DL:

* conv1: Camada convolutiva com 6 *features map* e um kernel de 5x5 por cada *features map*.
* pool1: Camada max-pooling com 6 *features map* e um kernel de 2x2 e stride de 2 por cada *features map*.
* conv2: Camada convolutiva com 16 *features map* e um kernel de 6x6 por cada *features map*.
* pool2: Camada max-pooling com 16 *features map* e um kernel de 2x2 e stride de 2 por cada *features map*.
* local3: Camada flat + Camada full-connected com 128 *features map*.
* local4: Camada full-connected com 84 *features map*.
* softmax: Camada softmax com 10 saídas (10 classes).

Com mini-batch = **100** e steps = **2000**.

Analisar os seguintes casos:

In [ ]:
%reset

In [ ]:
!rm -rf train

### Caso 1

Utilizar o otimizador SGD ([Stochastic gradient descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)) com taxa de aprendizagem 0.001 (optimizer='SGD').

In [ ]:
!cp Lenet5_model.py Lenet5_SGD_model.py

Abrir o arquivo [Lenet5_SGD_model.py](../../edit/Lenet5_SGD_model.py) e completar o código para ter a seguinte distribuição de camadas solicitada (editar os **FIXME** no arquivo).

Rodar o modelo:

In [ ]:
%run Lenet5_SGD_model.py

### Caso 2:

Copie o modelo já modificado Lenet5_SGD_model.py para Lenet5_RMS_model.py:

In [ ]:
!cp Lenet5_SGD_model.py Lenet5_RMS_model.py

No arquivo [Lenet5_RMS_model.py](../../edit/Lenet5_RMS_model.py), modificar o método de otimização para RMSProp:([Root Mean Square Propagation](http://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf)) com taxa de aprendizagem 0.001 (optimizer=tf.train.RMSPropOptimizer(learning_rate=0.001)).

**Opcional 1**: Podemos observar alguns resultados no *tensorboard* para abrir executar na consola:

tensorboard --logdir=train --port 8XXX

**Opcional 2**: Adicionar variaveis para o *tensorboard*

Para saber o IP de da maquina do servidor:

In [ ]:
!ifconfig

In [ ]:
!tensorboard --logdir=train --port 8111

# Keras

#### Modulos de keras

In [ ]:
import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.models import model_from_json
from keras_tqdm import TQDMNotebookCallback

#### Parametros da rede

In [ ]:
batch_size = 128
num_classes = 5
epochs = 5
img_rows, img_cols = 28, 28
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

#### Leitura de dados

In [ ]:
now = datetime.datetime.now

def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    
    
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=0,
              callbacks=[TQDMNotebookCallback()],
              validation_data=(x_test, y_test))
    
    score = model.evaluate(x_test, y_test, verbose=0)
    
    print('Training time: %s' % (now() - t))
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [ ]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

#### Criação do Modelo

In [ ]:
feature_layers = [
    Conv2D(36, kernel_size=(5, 5), activation='relu',input_shape=input_shape),
    MaxPooling2D(pool_size=(2, 2), strides=2),
    Conv2D(64, (6, 6), activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=2),
    Flatten()
]

classification_layers = [
    Dense(1024, activation='relu'),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
]

model = Sequential(feature_layers + classification_layers)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

#### Treinamento

In [ ]:
# create complete model
model = Sequential(feature_layers + classification_layers)

# train model for 5-digit classification [0..4]
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

# Transfer Learning

## Salvamos o Modelo (.json e .h5)

In [ ]:
# serializa modelo em formato JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serializa pesos pra HDF5
model.save_weights("model.h5")
print("Modelo armazenado no disco")

## Em outro momento....

In [ ]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Modelo cargado do disco")

## Treinando as camadas de classificação

In [ ]:
for l in feature_layers:
    l.trainable = False

In [ ]:
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)